In [1]:
import requests, re, json, datetime
import os, sys
from bs4 import BeautifulSoup as bs
from urllib.parse import urljoin
import uuid
from hashlib import md5
import dateparser
import pandas as pd


import ScrapeBakery as sb

In [2]:
now = datetime.datetime.now().timestamp()
a_day = 86400
days_since_start=1
start_timestamp = now-(a_day * days_since_start)
page = sb.IdeaPage("https://www.halfbakery.com/idea/Autoweeeebile", start_timestamp)

10c45666c06d3544ad71e2941f471c0d


In [81]:
first_time=False
# This makes use of a search url, returning all the ideas posted in some time period (defaulted to a day)
a_day = 86400
if first_time == True:
    days_since_start = (datetime.datetime.now() - datetime.datetime(2020,10,17) ).days
    
    start_timestamp = now-(a_day * days_since_start)
    guess_m = 500
    t_minus = int(now - start_timestamp)
    urls = ["https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:dn={m}:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(m=guess_m,t_minus=t_minus),
            "https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:do=100:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(t_minus=t_minus),
            "https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:do=200:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(t_minus=t_minus),
            "https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:do=300:dn=100:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(t_minus=t_minus)]
    
else:
    start_timestamp = now-(a_day * 1)
    guess_m = 100
    t_minus = int(now - start_timestamp)
    urls = ["https://www.halfbakery.com/view/ftm=r{t_minus}:s=Qr:d=irq:dn={m}:ds=0:n=Today_27s_20Notions:i=A_20list_20of_20todays_20ideas_20and_20annotations:t=Today_27s_20Notions".format(m=guess_m,t_minus=t_minus)]
    

data_harvest = sb.HarvestLinks(urls, start_timestamp)
print("Number of unique links to follow: {l}".format(l=len(data_harvest.fetch_links)))
data_harvest.fetch_data()

Number of unique links to follow: 16
Processing...
 16 pages to collect
d57aad695eb13761ed5db86ed75eacbe
fd3da03816cb3b8b0cf9f870c0e743c5
21b635c0c1b18f0156c74871c642c6fc
43e4757c44bbfb76132018b85f382d50
fbeb8c1ba52b192823899c8e98ea5d44
47ced7e4f63c39b232c43b3b6fab6fae
f390daf0a23fc211c05bf5e49586cc9f
c3eb9bca6c09992574f0cc3927390215
07712693fb4a7061de405c1839544e4a
d7633bfddce0c326da3cdf2b8ec7fdc8
Read 10 of 16...
85198a768c1f352458abc0b8a11635c8
b0f53819a83b6f79e87b3627a8f60917
42fbb19c4c8f2868a66573e50fcedb54
2245ac4906edf6c1398b09a76e6d9e1b
8b1b4abeff5f6a7e1a5a6698144a9eda
ac712348b06af6fb08391018d21df8f9

Complete read 16 of 16


In [4]:
ds = sb.DataStore(trash=False)

Building new schema any existing content will be lost.


In [5]:
ds.query_to_recordset("select tbl_name from sqlite_master")

[{'tbl_name': 'idea_fetch'},
 {'tbl_name': 'anno_fetch'},
 {'tbl_name': 'link_fetch'},
 {'tbl_name': 'fresh_ideas'},
 {'tbl_name': 'latest_user_content'}]

In [82]:
for idea in data_harvest.contents:
    ds.save_idea(idea)

Hashmatch - already cached
Saving.
Saving.
Hashmatch - already cached
Hashmatch - already cached
Saving.
Hashmatch - already cached
Saving.
Hashmatch - already cached
Saving.
Saving.
Saving.
Hashmatch - already cached
Saving.
Saving.
Saving.


In [87]:
pd.DataFrame(ds.query_to_recordset("select (select count(*) from fresh_ideas) fresh, (select count(*) from idea_fetch) raw"))

,fresh,raw
0,302,310


In [15]:
contribution_df = pd.DataFrame(ds.query_to_recordset("select * from latest_user_content")).sort_values(by=["date","seq"])

In [103]:
match_d = {"Hitler" : [re.compile(r"(?i)\b(hitler)\b"), 
                       re.compile(r"(?i)\b(führer|fuhrer)\b"),
                       re.compile(r"(?i)\b(Adolph)\b")
                      ],
           "Idi Amin" : [re.compile(r"(?i)\b(Idi Amin)\b")],
           "Genghis Khan" : [re.compile(r"(?i)\b(Genghis Khan)\b")],
           "Donald Trump" : [re.compile(r"(?i)\b(Donald Trump|Trump)\b")],
           "Brexit" : [re.compile(r"(?i)\b(Brexit|remoaner)\b")],
           "Political Correctness" : [re.compile(r"(?i)\b(social justice|sjw|cancel culture|political correctness)\b")]
           
          }
def multi_matcher(text, match_d):
    matches={}
    wbs=[]
    wb_regex=re.compile(r"(\b)")
    for m in wb_regex.finditer(text):
        if m is not None:
            wbs.append(m.span()[0])
    for k,v in match_d.items():
        matches[k]=[]
        for vv in v:
            m = vv.finditer(text)
            if m is not None:
                for mg in m:
                    sp=mg.span()
                    try:
                        start_pos = [w for w in wbs if w < sp[0]-10][-1]
                    except IndexError:
                        start_pos = 0   
                    try:
                        end_pos = [w for w in wbs if w > sp[1]+10][0]
                    except IndexError:
                        end_pos = len(text)
                    matches[k].append ((sp, "//" + text[start_pos:end_pos] + "//"))
    for k in matches.keys():
        if matches[k]==[]:
            matches[k]=None
    matches = {k:v for k,v in matches.items() if v is not None}
    return matches

In [104]:
hmatch = contribution_df['text'].apply(lambda x : multi_matcher(x, match_d))

In [105]:
contribution_df[hmatch.apply(lambda x : x != {})]

,url,fetch_id,date,user,ctype,seq,text
933,https://www.halfbakery.com/idea/The_20Decider,a77e662f-cb87-4f5b-9609-f8efd950bd1f,1.322870e+09,theircompetitor,link,3,Trump set to moderate GOP debate
1296,https://www.halfbakery.com/idea/Trump_20Oddity,8abb0b96-cf0b-492c-8294-a545ad6ff895,1.488499e+09,popbottle,link,0,Springtime for Hitler and Germany
1298,https://www.halfbakery.com/idea/Trump_20Oddity,8abb0b96-cf0b-492c-8294-a545ad6ff895,1.488499e+09,RayfordSteele,anno,6,"America was having trouble\r\nWhat a sad, sad..."
1305,https://www.halfbakery.com/idea/Trump_20Progre...,c0635239-c678-4f68-a59e-d71d849af1cd,1.490656e+09,DenholmRicshaw,idea,-1,"Project on a suitable erection, say a large ho..."
1309,https://www.halfbakery.com/idea/Trump_20Progre...,c0635239-c678-4f68-a59e-d71d849af1cd,1.490742e+09,MaxwellBuchanan,anno,3,//Trump Progress Bar// Mine's a large one.
1330,https://www.halfbakery.com/idea/Trump_20Progre...,c0635239-c678-4f68-a59e-d71d849af1cd,1.491520e+09,not_morrison_rm,link,1,Twitter users scalded Trump
1344,https://www.halfbakery.com/idea/Trump_20Progre...,c0635239-c678-4f68-a59e-d71d849af1cd,1.492988e+09,xenzag,anno,27,Apparently now there is person referred to as...
1404,https://www.halfbakery.com/idea/Hijack_20the_2...,e8f4d1e9-bae8-458a-a2ca-4329dc22889b,1.516493e+09,DDRopDeadly,anno,0,"Like it, maybe the powers that be would have ..."
1469,https://www.halfbakery.com/idea/The_20universa...,b878a43e-000a-4a11-b122-32e9d3ca72ea,1.518653e+09,doctorremulac3,anno,49,"//So, what's the standing of egalité and frat..."
1556,https://www.halfbakery.com/idea/Trumphemian_20...,c199b3c5-b904-4c1d-bfc1-6c11a7665ecc,1.553040e+09,doctorremulac3,anno,20,"//Easy come, easy go, will you build my wall ..."


In [135]:
for k in match_d.keys():
    matches=hmatch.apply(lambda x : k in x.keys())
    kmatches=hmatch[matches].apply(lambda x : x.get(k))
    not_exclusions = contribution_df["url"].apply(lambda x : x not in ["https://www.halfbakery.com/idea/F_fcrst_20annual_20HalfBakery_20_93Wo_20ist_20der_20F_fchrer_20_3f_94_20programming_20competition_2e", "https://www.halfbakery.com/idea/Days_20Since_20Hitler_20Was_20Mentioned_20Here"])
    findex=((matches) & (not_exclusions))
    f_df = contribution_df[findex]
    f_df['matches']=kmatches
    print()
    try:
        final_mention = f_df.loc[f_df.groupby("date").seq.idxmax()].iloc[-1]
        #penultimate_mention = f_df.loc[f_df.groupby("date").seq.idxmax()].iloc[-2]
        #last_period = (final_mention['date']-penultimate_mention['date'])/86400
        mention_period = int((now - final_mention['date'])/86400)
        t = ds.query_to_recordset("select distinct title from idea_fetch where url=?",[final_mention['url']])
        #print(t[0]['title'])
        
        print("It has been {d} days since {t} was last mentioned.".format(d=int(mention_period),t=k))
        print("{c} found on {l} by [{u}] {t} on {d}".format(c=final_mention['ctype'], l=t[0]['title'], u=final_mention['user'], t=final_mention['matches'][0][1].replace("\n", "").replace("\r", ""), d=datetime.datetime.fromtimestamp(final_mention['date']).strftime("%d %b %Y")))
        #print ( final_mention )
    except IndexError as err:
        #print("{t}\t".format(t=k), err)
        print("No mention of {t} found in the cache.".format(t=k))


            


It has been 4 days since Hitler was last mentioned.
anno found on Fishties by [zen_tom] //to Adolph Hitler caused me // on 17 Dec 2020

No mention of Idi Amin found in the cache.

No mention of Genghis Khan found in the cache.

It has been 5 days since Donald Trump was last mentioned.
anno found on MentorBubbles; view the internet versions of the people who are doing well by [xenzag] //someone like Trump gets endless// on 16 Dec 2020

It has been 19 days since Brexit was last mentioned.
idea found on Swap populations of Scotland and Northern Ireland by [hippo] //Post-Brexit, when the // on 02 Dec 2020

It has been 14 days since Political Correctness was last mentioned.
anno found on Big Brother's Mind Numbing Tea Biscuits by [zen_tom] //compliance and political correctness, parroting// on 07 Dec 2020


/home/tomk/environment/py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [108]:
f_df.loc[f_df.groupby("date").seq.idxmax()].iloc[-1]

url         https://www.halfbakery.com/idea/Big_20Brother_...
fetch_id                 91e658f6-c35d-41bf-9816-9524c6a122fe
date                                               1.6073e+09
user                                                  zen_tom
ctype                                                    anno
seq                                                        20
text         Look, I see a bunch of tribal ideas about com...
matches     [((58, 79), //compliance and political correct...
Name: 5249, dtype: object

86400.0